# Quantum Reservoir Computing Demo

This notebook shows a tiny example of quantum reservoir computing (QRC) and a classical reservoir baseline. All figures render as SVG so they stay friendly to Git diffs.

In [ ]:
%config InlineBackend.figure_formats = ['svg']
import numpy as np
import matplotlib.pyplot as plt
from app import dataset
from app.quantum_reservoir import QuantumReservoir
from app.classical_reservoir import ClassicalReservoir
from app.readout import train_readout, predict_readout
from app.utils import standardize, destandardize
from sklearn.metrics import mean_squared_error


## 1. Build a small dataset
We use a noisy sine wave and create sliding windows for next-step prediction.

In [ ]:
sequence = dataset.generate_sine_sequence(n_points=120, noise=0.05, seed=0)
X, y = dataset.build_supervised_pairs(sequence, window_size=5)
X_train, X_test, y_train, y_test = dataset.train_test_split(X, y, test_size=0.25, seed=0)
y_train_std, mean, std = standardize(y_train)
y_test_std = (y_test - mean) / (std + 1e-8)


## 2. Quantum reservoir encoding
We keep the circuit small so it runs quickly on CPU.

In [ ]:
qrc = QuantumReservoir(n_qubits=2, depth=2, seed=0)
R_q_train = qrc.encode_sequence(X_train)
w_q = train_readout(R_q_train, y_train_std)
R_q_test = qrc.encode_sequence(X_test)
pred_q = predict_readout(R_q_test, w_q)
pred_q = destandardize(pred_q, mean, std)
mse_q = mean_squared_error(y_test, pred_q)
mse_q


## 3. Classical reservoir baseline

In [ ]:
classic = ClassicalReservoir(n_inputs=X_train.shape[1], n_reservoir=30, seed=0)
R_c_train = classic.encode_sequence(X_train)
w_c = train_readout(R_c_train, y_train_std)
R_c_test = classic.encode_sequence(X_test)
pred_c = predict_readout(R_c_test, w_c)
pred_c = destandardize(pred_c, mean, std)
mse_c = mean_squared_error(y_test, pred_c)
mse_c


## 4. Compare predictions
We plot the target sequence against both models. All plots save as SVG for easy viewing on GitHub.

In [ ]:
fig, ax = plt.subplots(figsize=(7,3))
ax.plot(y_test, label='Target', color='black')
ax.plot(pred_q, label='Quantum RC', linestyle='--')
ax.plot(pred_c, label='Classical RC', linestyle=':')
ax.legend()
ax.set_xlabel('Time step')
ax.set_ylabel('Value')
fig
